In [ ]:
import pandas as pd
import warnings
import re
import tables
import mysql.connector
warnings.filterwarnings('ignore')

In [ ]:
def abre_conexao_banco_de_dados():
  global mydb
  mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

In [ ]:
def retorna_tabelas_banco_de_dados(nomeTabela):
  if nomeTabela =='genealogia_embrapa':
    sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
  else:
    sql = f"SELECT * FROM {nomeTabela}"
  df =  pd.read_sql(sql, mydb)
  return df

In [ ]:
def cria_df_com_somente_os_dados_novos(planilhaAntiga, planilhaNova):
  dados_em_comum = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
  dados_em_comum = dados_em_comum.drop(dados_em_comum.filter(regex='_drop$').columns, axis=1)
  dados_em_comum = dados_em_comum.drop_duplicates()
  df_merge = pd.merge(dados_2023, dados_em_comum, how='left', indicator=True, suffixes=('', '_drop'))
  # filtra somente as linhas que estão no dataframe da esquerda
  df_dados_novos = df_merge[df_merge['_merge'] == 'left_only']
  # remove a coluna "_merge"
  df_dados_novos.drop('_merge', axis=1, inplace=True)
  df_dados_novos = df_dados_novos.drop(df_merge.filter(regex='_drop$').columns, axis=1)
  df_dados_novos = df_dados_novos.drop_duplicates()
  return df_dados_novos

In [ ]:
def adiciona_coluna_gs_aos_dados_novos(df,df_pedigree ):
  df_temporaria = pd.merge(df, df_pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
  df=df_temporaria
  nome_antigo = 'SEXO'
  nome_novo = 'sexo'
  indice_coluna = df.columns.get_loc(nome_antigo)
  df.rename(columns={nome_antigo: nome_novo}, inplace=True)
  return df

In [ ]:
def retira_acentos_espacos(df):
  # remoção dos hifens e espaços em branco da coluna RGVACA
  df.loc[df['COMPRAC'] == 11, 'RGVACA'] = df.loc[df['COMPRAC'] == 11, 'RGVACA'].str.replace('-', '').str.replace(' ', '')

  df.loc[df['GSMAE'] == 11, 'RGMAE'] = df.loc[df['GSMAE'] == 11, 'RGMAE'].str.replace('-', '').str.replace(' ', '')

  df.loc[df['GSPAI'] == 11, 'RGPAI'] = df.loc[df['GSPAI'] == 11, 'RGPAI'].str.replace('-', '').str.replace(' ', '')
  return df

In [ ]:
def adiciona_colunas_codprod_codpai_cod_mae(df, genealogia_abcz):
  sub_tabela_gs_0 = df[df['gs'] == 0]
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  df_temporaria1 = pd.merge(sub_tabela_gs_0, genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
  sub_tabela_gs_0=df_temporaria1
  sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()

  df_temporaria4 = pd.merge(df, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
  df=df_temporaria4
  df = df.drop_duplicates()
  return df

In [ ]:
def pegar_codpai_codmae_abcz_rg_3_letras(df, genealogia_abcz):

  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela para maes
  subtabela_mae = df[df['GSMAE'] == 11]
  #e o cod_mae e nulo
  subtabela_mae
  # selecionar apenas as linhas que começam com letras
  subtabela_mae = subtabela_mae[subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Buscando cod_mae na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna cod_prod_mae no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela em que gspai = 11
  subtabela_pai = df[df['GSPAI'] == 11]

  # selecionar apenas as linhas que começam com letras
  subtabela_pai = subtabela_pai[subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na ABCZ
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Criando coluna do cod_prod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando valores da cod_pai e cod_mae do df com os valores das colunas inseridas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')

  return df


In [ ]:
def pegar_codpai_codmae_abcz_rg_diferente_3_letras(df, genealogia_abcz):
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela
  subtabela_pai = df[df['GSPAI'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_pai = subtabela_pai[~subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_pai_gen'})
  subtabela_pai = subtabela_pai.drop_duplicates()
  genealogia_abcz = genealogia_abcz.drop_duplicates()
  subtabela_pai['nome4caracteres'] = subtabela_pai['nomep'].str.slice(start=0, stop=4)
  genealogia_abcz['nome4caracteres'] = genealogia_abcz['nome'].str.slice(start=0, stop=4)
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz[['RGVACA', 'nome4caracteres','cod_prod_pai_gen']], left_on=['RGPAI', 'nome4caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela mae
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = df[df['GSMAE'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_mae = subtabela_mae[~subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_mae na abcz
  subtabela_mae['nome11caracteres'] = subtabela_mae['nomem'].str.slice(start=0, stop=11)
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
  #criando coluna com os 10 caracteres do nome
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz[['RGVACA','nome4caracteres' ,'cod_prod_mae_gen']], left_on=['RGMAE', 'nome11caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_mae_gen no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando cod_pai e cod_mae com base nas novas colunas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')
  return df

In [17]:
def adiciona_coluna_reb(df, criadores):
  criadores = criadores.rename(columns={'codgiro':'reb_giro'})
  df = pd.merge(df, criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
  df = df.drop_duplicates()
  return df

In [ ]:
abre_conexao_banco_de_dados()
df_genealogia_embrapa = retorna_tabelas_banco_de_dados('genealogia_embrapa')
df_genealogia_embrapa =df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = retorna_tabelas_banco_de_dados('genealogia_abcz')
df_criadores = retorna_tabelas_banco_de_dados('criadores')
df_producao = retorna_tabelas_banco_de_dados('producao_embrapa')
df_fazendas = retorna_tabelas_banco_de_dados('fazendas')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [ ]:
df_dados_novos = cria_df_com_somente_os_dados_novos(dados_2022, dados_2023)

In [ ]:
df_dados_novos = adiciona_coluna_gs_aos_dados_novos(df_dados_novos, pedigree)

In [ ]:
df_dados_novos = retira_acentos_espacos(df_dados_novos)

In [ ]:
df_dados_novos =adiciona_colunas_codprod_codpai_cod_mae(df_dados_novos, df_genealogia_abcz)

In [ ]:
df_dados_novos = pegar_codpai_codmae_abcz_rg_3_letras(df_dados_novos, df_genealogia_abcz)

In [ ]:
df_dados_novos = pegar_codpai_codmae_abcz_rg_diferente_3_letras(df_dados_novos, df_genealogia_abcz)

In [18]:
df_dados_novos = adiciona_coluna_reb(df_dados_novos, df_criadores)

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2,74612019122,NaN,5,F,NaN,NaN,NaN,1291
2,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,10639-GIRO
3,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN
4,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,37634-ABCZ
5,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,2,60292021222,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ
1151,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,2,1322019222,NaN,6,F,NaN,NaN,NaN,31835-ABCZ
1152,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,2,69152021222,NaN,5,F,NaN,NaN,NaN,27729-ABCZ
1153,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2,2,60292019122,NaN,5,F,NaN,NaN,NaN,31365-ABCZ
